In [2]:
import re, yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from bokeh.plotting import figure, show, output_notebook, curdoc
from bokeh.io import push_notebook
from bokeh.models import ColumnDataSource, HoverTool, TapTool, CustomJS, Div
from bokeh.layouts import column
from bokeh.themes import Theme
from bokeh.document import Document
from bokeh.embed import file_html, components
from bokeh.util.browser import view
from bokeh.models import Circle

# Load saved dataframe

# next steps

1. create a topic filter/buttons to get representative docs for each topic of choice 
3. save dataframe with text and embeddings for easy searching
4. try a subsection of english texts 
5. add semantic search / transform > "what would johannes say about..."

In [3]:
df_pickle = pd.read_pickle("df_topic_model_all")

In [302]:
df_pickle['texts'][df_pickle['Topic'] == 3].sample(2).values.tolist()

['Is facebook een sedimentatie van je leven of een profiel?\n',
 'Voor kortverhaal: moeten overheidsfuncties vervangen worden door bedrijven als facebook? Of moeten ze zelf websites maken?\n']

In [4]:
df_pickle.texts.tolist()
df_pickle['2d_embeddings_x'].tolist()
df_pickle['2d_embeddings_y'].tolist()
df_pickle.Name.unique().tolist()

['14_taal_spreken_discours_woorden',
 '-1_onze_mensen_wel_wereld',
 '34_drugs_mdma_jake_persoon',
 '18_overtuigen_gedrag_denken_kritiek',
 '0_wetenschap_intelligentie_onze_reality',
 '12_reizen_droom_reis_travel',
 '20_lage_opvattingen_erkenning_hoog',
 '31_wir_feestje_dronken_manufacturing',
 '17_empathy_empathie_anderen_schaamte',
 '19_personage_personages_verteller_schouwspel',
 '9_blog_blogs_johannes_eerste',
 '6_voelde_kamer_bed_ogen',
 '21_angst_emoties_pijn_zorgen',
 '1_geld_sociale_sociaal_ethische',
 '39_amp_online_netlog_http',
 '7_porno_seks_seksuele_masturberen',
 '44_narcisme_jezelf_mezelf_sympathiek',
 '4_muziek_dj_music_muzikale',
 '5_europa_frankrijk_europese_land',
 '22_leven_verleiding_overgrootouders_orde',
 '33_netwerken_netwerk_mensen_groep',
 '8_boek_schrijver_schrijven_lezen',
 '13_droom_dromen_slaap_denken',
 '11_stad_road_car_cyclists',
 '42_kind_gemasseerd_geborgenheid_wij',
 '16_internet_zoeken_netwerk_verkeerde',
 '46_culturen_cultuur_identiteit_whitehouse',

In [5]:
df_pickle['keywords'] = df_pickle.Top_n_words.apply(lambda x: ', '.join([word.strip() for word in x.split('-')]))

In [167]:
# unused
texts = [text[:1000] for text in df_pickle.texts.tolist()]

texts = [text + "... " if len(text) > 999 else text for text in texts]
# print(texts[:5])

['Wat ik hier wil doen is, net als in mijn dagboek van een auteur, een discours ontwikkelen voor een boek. Hier zal de nadruk minder komen te liggen op ‘onderliggende ideeën’ maar eerder op de oppervlakten van de taal zelf. Concreet wil ik het alledaagse taalgebruik onderzoeken, de taal van het denken en spreken, de taal van mijmeringen, redeneringen, dialogen... Ik wil ze vanuit verschillende personages onderzoeken. Personages spreken en denken. De taal is het middelpunt.\nIk wil voornamelijk plaats geven aan opvallendheden die contrasten karakteriseren tussen hoe verschillende concrete personen denken, verantwoordingen afleggen, en tot handelen gemotiveerd worden. Ik wil hier zodoende enkele personages vormgeven. Het wordt een soort tussenstadium van de bedenkingen die ik me maak over het denken van anderen en het schrijven vanuit het perspectief van de personages.\nDe wijze waarop ik dit wil aanvatten is nu sterk beïnvloed door Joyce. Maar het gaat hier om experimenteren met dialoog

In [6]:
# inintuitive but good: np.isin gives True or False 
mask = np.isin(df_pickle.Topic, [-1], invert=True)

In [7]:
mycolors = np.array(list(mcolors.TABLEAU_COLORS.values()) + \
            [mcolors.to_hex(c) for c in plt.cm.Pastel1.colors] + \
            [mcolors.to_hex(c) for c in plt.cm.Pastel2.colors] + \
            [mcolors.to_hex(c) for c in plt.cm.Paired.colors]
            )

source = ColumnDataSource(data=dict(
    x=df_pickle['2d_embeddings_x'].to_numpy()[mask],
    y=df_pickle['2d_embeddings_y'].to_numpy()[mask],
    colors=mycolors[np.array(df_pickle.Topic[mask]) % len(mycolors)],
    onderwerp=[s[:80] + "..." for s in df_pickle.texts.to_numpy()[mask]],
    description=[s[:500] + "..." for s in df_pickle.texts.to_numpy()[mask]], # instead of the unused shortening
    keywords=np.array(df_pickle['keywords'][mask])[np.array(df_pickle.Topic[mask])]
    ))

In [36]:
hover = HoverTool()
hover.tooltips = [
    ("Topic colour", "$color:mycolors"),
                  ("Topic keywords", "@keywords")]

# """
# <div style="max-width: 150px; border: none; box-shadow: none;">
#     <span style="font-size: 10px; color: #666;">@keywords</span>
# </div>
# """


description_div = Div(text="", width=900, height=100)

callback = CustomJS(args=dict(source=source, div=description_div), code="""
    const indices = source.selected.indices;
    if (indices.length == 0) {
        div.text = "";  // Clear the div if no point is selected
        return;
    }
    const desc = source.data['description'][indices[0]];
    div.text = desc;  
""")

In [244]:
yaml_string = """
attrs:
    Plot:
        background_fill_color: !!null 
        border_fill_color: !!null 
        outline_line_color: !!null 

    Axis:
        axis_line_color: !!null  
        major_tick_line_color: !!null 
        minor_tick_line_color: !!null
        major_label_text_color: !!null 
        axis_label_standoff: 0  

    Grid:
        grid_line_color: !!null

    Legend:
        background_fill_color: !!null  
        border_line_color: !!null

    Title:
        text_color: !!null
"""

theme = Theme(json=yaml.safe_load(yaml_string))

In [37]:
output_notebook()

curdoc().theme = theme

plot = figure(tools="wheel_zoom, reset", 
              title="Johannes' universe", 
              width=960, height=700)

plot.circle('x', 'y', source=source, color='colors', fill_alpha=0.2, size=10)
plot.add_tools(hover)
plot.add_tools(TapTool(callback=callback))

layout = column(plot, description_div, sizing_mode="scale_both") 

curdoc().add_root(layout)

show(layout, notebook_handle=True)

Loading BokehJS ...

In [179]:
filename = "plot.html"
with open(filename, "w") as f:
        f.write(file_html(layout, title=plot.title.text))
print(f"Wrote {filename}")
view(filename)

Wrote plot.html


In [172]:
# curdoc().clear()

# Add a selection box at the bottom

This could be useful to let users choose a topic and generate a blog based on random items in the topic 

In [174]:
from bokeh.models.widgets import Select

# Unique keywords from your data source
unique_keywords = list(set(source.data['keywords']))

# Create a selection box
keyword_select = Select(title="Choose a Topic:", value=unique_keywords[0], options=unique_keywords)


In [175]:
update_callback = CustomJS(args=dict(source=source, div=description_div, select=keyword_select), code="""
    // Get the selected keyword
    var selected_keyword = select.value;
    
    // Find the description that matches the keyword (you might need to adjust this logic)
    for (var i = 0; i < source.data.keywords.length; i++) {
        if (source.data.keywords[i] === selected_keyword) {
            div.text = source.data.description[i];
            break;
        }
    }
""")

keyword_select.js_on_change('value', update_callback)


In [178]:
from bokeh.layouts import column

# Creating a layout
layout = column(plot, description_div, keyword_select)

# Add the layout to the current document
curdoc().add_root(layout)

# invisible glyphs for cluster zoom

In [106]:
# cluster centers
centroids = df_pickle.groupby('Topic').agg({
    '2d_embeddings_x': 'mean',
    '2d_embeddings_y': 'mean'
}).reset_index()

# Rename the columns to reflect that they are centroids.
centroids.columns = ['Topic', 'Centroid_x', 'Centroid_y']

In [100]:
# Ignore - Radii
def calculate_radius(group, centroid_x, centroid_y):
    distances = np.sqrt((group['2d_embeddings_x'] - centroid_x) ** 2 + (group['2d_embeddings_y'] - centroid_y) ** 2)
    return np.mean(distances)

# Calculate the radii for each cluster
df_pickle_ = df_pickle[df_pickle['Topic'] > -1].copy()
radii = df_pickle_.groupby('Topic').apply(lambda group: calculate_radius(group, 
                                                                       df_pickle.loc[group.name, '2d_embeddings_x'], 
                                                                       df_pickle.loc[group.name, '2d_embeddings_y']))

# Add the radii to the centroids DataFrame
centroids.loc[centroids['Topic'] > -1, 'Radius'] = 0.5 # radii.values

In [83]:
# get rid of -1
centroids = centroids.loc[centroids['Topic'] > -1]

In [121]:
# to get the names
match_maker = df_pickle[['Topic', 'Name']].drop_duplicates()
centroids = centroids.merge(match_maker, on='Topic', how='left')
centroids.head()

,Topic,Centroid_x,Centroid_y,Name
0,-1,3.741693,1.094679,-1_onze_mensen_wel_wereld
1,0,3.309287,1.160651,0_wetenschap_intelligentie_onze_reality
2,1,1.774688,-0.874522,1_geld_sociale_sociaal_ethische
3,2,6.412583,-0.795670,2_liefde_love_relatie_feel
4,3,1.662565,4.582716,3_facebook_fb_bda_vanavond


In [ ]:
centroids_['Name'] = 

In [235]:
cluster_source.data['topic']

['-1_onze_mensen_wel_wereld',
 '0_wetenschap_intelligentie_onze_reality',
 '1_geld_sociale_sociaal_ethische',
 '2_liefde_love_relatie_feel',
 '3_facebook_fb_bda_vanavond',
 '4_muziek_dj_music_muzikale',
 '5_europa_frankrijk_europese_land',
 '6_voelde_kamer_bed_ogen',
 '7_porno_seks_seksuele_masturberen',
 '8_boek_schrijver_schrijven_lezen',
 '9_blog_blogs_johannes_eerste',
 '10_impact_social_business_innovatie',
 '11_stad_road_car_cyclists',
 '12_reizen_droom_reis_travel',
 '13_droom_dromen_slaap_denken',
 '14_taal_spreken_discours_woorden',
 '15_beelden_kunst_beeld_taal',
 '16_internet_zoeken_netwerk_verkeerde',
 '17_empathy_empathie_anderen_schaamte',
 '18_overtuigen_gedrag_denken_kritiek',
 '19_personage_personages_verteller_schouwspel',
 '20_lage_opvattingen_erkenning_hoog',
 '21_angst_emoties_pijn_zorgen',
 '22_leven_verleiding_overgrootouders_orde',
 '23_klimmen_wanneer_berg_meter',
 '24_seks_meisjes_steffen_erotiek',
 '25_mannen_vrouwen_mannelijke_vrouw',
 '26_moraal_traditie_le

In [255]:
curdoc().theme

TypeError: Theme.apply_to_model() missing 1 required positional argument: 'model'

In [311]:
# reset
curdoc().clear()

curdoc().theme = theme # "night_sky"
mycolors = np.array(list(mcolors.TABLEAU_COLORS.values()) + \
            [mcolors.to_hex(c) for c in plt.cm.Pastel1.colors] + \
            [mcolors.to_hex(c) for c in plt.cm.Pastel2.colors] + \
            [mcolors.to_hex(c) for c in plt.cm.Paired.colors]
            )

source = ColumnDataSource(data=dict(
    x=df_pickle['2d_embeddings_x'].to_numpy()[mask],
    y=df_pickle['2d_embeddings_y'].to_numpy()[mask],
    colors=mycolors[np.array(df_pickle.Topic[mask]) % len(mycolors)],
    # onderwerp=[s[:80] + "..." for s in df_pickle.texts.to_numpy()[mask]],
    # description=[s[:500] + "..." for s in df_pickle.texts.to_numpy()[mask]], # instead of the unused shortening
    # keywords=np.array(df_pickle['keywords'][mask])[np.array(df_pickle.Topic[mask])]
    ))

# Create a data source for the clusters
cluster_source = ColumnDataSource(data=dict(
        x=centroids['Centroid_x'].tolist(),
        y=centroids['Centroid_y'].tolist(),
        topic_name=centroids['Name'].tolist(),
        topic=centroids['Topic'].tolist()
    ))

plot = figure(tools="wheel_zoom, reset", 
              title="Johannes' universe", 
              width=960, height=700)

plot.circle('x', 'y', source=source, color='colors', fill_alpha=0.2, size=10)

# Add invisible circles for each cluster
invisible_circles = Circle(x='x', y='y', radius=0.5, fill_alpha=0, line_color=None, fill_color='white')
visible_circles = Circle(x='x', y='y', radius=0.5, fill_alpha=0.05, line_color=None, fill_color='white')

renderer = plot.add_glyph(cluster_source, invisible_circles, hover_glyph=visible_circles)
renderer.nonselection_glyph = Circle(x='x', y='y', radius=0.5, fill_alpha=0, line_color=None, fill_color='white')

cluster_hover = HoverTool(point_policy="follow_mouse")
cluster_hover.renderers = [renderer]
cluster_hover.tooltips = None # [("topic", "@topic_name")]
plot.add_tools(cluster_hover)

# scatter_hover = HoverTool(point_policy="follow_mouse")
# scatter_hover.tooltips = None # [("topic", "@topic_name")]
# plot.add_tools(scatter_hover)

# taptool 
# description_div = Div(text="") #, width=900, height=10)

cluster_callback = CustomJS(args=dict(cluster_source=cluster_source, 
                                      source=source, plot=plot,
                                    #   div=description_div, 
                                    #   scatter_hover=scatter_hover,
                                      cluster_hover=cluster_hover
                                      ), code="""
                            
    function zoomToCluster(cluster_center_x, cluster_center_y) {
        // Adjust the plot range to center on the cluster
        plot.x_range.start = cluster_center_x + 0.8;
        plot.x_range.end = cluster_center_x - 0.8;
        plot.y_range.start = cluster_center_y + 0.8;
        plot.y_range.end = cluster_center_y - 0.8;
        }
        
    // Check if a cluster circle was tapped
    var cluster_hit_test = cluster_source.selected.indices;
    var scatter_hit_test = source.selected.indices;
    if (cluster_hit_test.length == 0) {
        cluster_hover.tooltips = null;
        // scatter_hover.tooltips = null; // hide the tooltips if no cluster is selected
        // div.text = "";  // Clear the div if no cluster is selected
        // return;
    }
    if (cluster_hit_test.length > 0) {
        var selected_index = cluster_hit_test[0];
        var cluster_center_x = cluster_source.data['x'][selected_index];
        var cluster_center_y = cluster_source.data['y'][selected_index];
        
        // scatter_hover.tooltips = `<div style="max-width: 150px;">
        // <span style="font-size: 10px; color: #666;">@onderwerp</span>
        // </div>`;
        
        zoomToCluster(cluster_center_x, cluster_center_y);
        
        // Send the topic number to a flask endpoint 
        const topic = cluster_source.data['topic'][selected_index];
        console.log('Zoomed to cluster topic:', topic);
        generateTopic(topic);
        
        // const topicName = cluster_source.data['topic_name'][selected_index];
        // Topic: ${topicName}<br><br>
        
        // div.text = `<button onclick="generateText('${topic}')">click here to generate text.</button>`;
        
        } else {
        console.log('No cluster circle was tapped');
     }
    // Check if a scatter point was tapped
    // if (scatter_hit_test.length > 0) {
        // var scatter_index = scatter_hit_test[0];
        // const desc = source.data['description'][scatter_index];
        // div.text = desc; 
        // console.log('a scatter point was tapped');
    // }       
""")

# Add the TapTool with the callback
tap_tool = TapTool(callback=cluster_callback)
plot.add_tools(tap_tool)

layout = column(children=[plot, description_div], sizing_mode="scale_width", name="layout") 
# Add the layout to the current document
curdoc().add_root(layout)

curdoc().validate()

In [280]:
output_notebook()
show(layout, notebook_handle=True)

Loading BokehJS ...

```py
filename = "plot.html"
with open(filename, "w") as f:
        f.write(file_html(layout, title=plot.title.text))
print(f"Wrote {filename}")
view(filename)
```

In [5]:
script, div = components(layout)
print(script, div)

NameError: name 'layout' is not defined

In [ ]:
df_pickle = pd.read_pickle("df_topic4.pkl")

df_pickle.groupby('Topic')['Top_n_words'].first().tolist()

In [32]:
df_pickle.columns

Index(['Topic', 'Name', 'Top_n_words', '2d_embeddings_x', '2d_embeddings_y',
       'texts', 'mask', 'embeddings'],
      dtype='object')

In [35]:
df_pickle.groupby('Topic')['Top_n_words'].first().tolist()

['onze - mensen - wel - wereld - leven - waar - enkel - nieuwe - gaan - gaat',
 'wetenschap - intelligentie - onze - reality - filosofie - dingen - kennis - world - waarheid - wereld',
 'geld - sociale - sociaal - ethische - moeten - economische - economist - systeem - markt - economie',
 'liefde - love - relatie - feel - romantische - feeling - dont - eenzaamheid - leven - would',
 'facebook - fb - bda - vanavond - vrienden - online - sociale - mensen - delen - foto',
 'muziek - dj - music - muzikale - dansen - bewegingen - techno - smaak - genre - nieuwe',
 'europa - frankrijk - europese - land - belgie - duitsland - amerikanen - geschiedenis - landen - history',
 'voelde - kamer - bed - ogen - anders - wit - vuur - oor - zoem - hoor',
 'porno - seks - seksuele - masturberen - rommel - beelden - orde - pornografische - norm - masturbatie',
 'boek - schrijver - schrijven - lezen - verhaal - mensen - gelezen - boeken - leven - moeten',
 'blog - blogs - johannes - eerste - rond - persoo

In [64]:
topic_names = ['nieuwe wereld',
 'wetenschap, filosofie, kennis, intelligentie, waarheid, wereld',
 'geld, ethisch, systeem, markt, economie, sociaal',
 'liefde, relatie, romantische, feeling, eenzaamheid, leven',
 'facebook, vanavond, sociale mensen, vrienden, delen',
 'muziek, dj, dansen, bewegen, smaak, genre',
 'europa, geschiedenis, landen, history, frankrijk, belgie',
 'kamer, bed, ogen, vuur, oor, zoem',
 'rommel, beelden, orde, norm, sensualiteit',
 'boek, schrijver, lezen, verhaal, mensen',
 'blog, persoonlijke, schrijven, hyperlinks, zin',
 'social, impact, business, innovatie, esg, people, change',
 'stad, cyclists, cars, london, public, space, traffic, transport',
 'reizen, droom, vlieger, avontuur, verhaal, vliegers, hoogtepunt',
 'droom, slaap, denken, nachtelijk',
 'taal, spreken, discours, woorden, vertaalbaarheid, denken',
 'beelden, kunst, taal, foto, representatie, beeldcultuur, artistiek, esthetica',
 'internet, netwerk, spiritueel, online, bereikbaar',
 'empathy, schaamte, gelukkig, leven, feeling',
 'overtuigen, kritiek, oordelen, grand, spreken, weerlegging',
 'personage, verteller, schouwspel, drama, perspectief, verhaal',
 'erkenning, concurrentie, voldoening, werk',
 'angst, emoties, pijn, zorgen, humeur, emotie',
 'leven, verleiding, overgrootouders, hoop, transcendente, spel',
 'klimmen, berg, natuur, zon, beneden, wand',
 'meisjes, erotiek, lippen, kus',
 'vrouwen, oppressie, man',
 'moraal, traditie, nihilisme, school, overtuiging, geloof, moreel',
 'mannen, vrouw, homo, kussen, hetero, meisjes',
 'vrienden, beleefd, interesses, kennissen, vriendelijk, ontmoetingen, gepraat',
 'ismael, ahabs, verteller, opinies',
 'informatie, downloaden, reclame, marketing, klanten, baas, adverteerders',
 'feestje, dronken, manufacturing, date, spontaan, alcohol, drunk, wasted',
 'fictie, wensen, beelden, wereld, verbeelding, wensdromen, werkelijkheid, openen',
 'netwerken, netwerk, mensen, groep, interesse, profiel, sociale, reputatie, leden,',
 'middelen, discours, roes, media, idolatry',
 'couchsurfing, reizen, gastheer, vrienden, concreet, profielen',
 'argentinie, espanol, voy, amigos, geloof',
 'hipsters, bobo, anti, kledij, geassimileerd, stereotypes, culturele, mainstream',
 'architectuur, kwaliteit, foto, architecten, architecturale, visuele, toerist, aandacht',
 'ainternet, vimeo, schaamte, persoonlijk',
 'god, theisme, pan, goddelijke, organisaties, religie',
 'spiegel, ogen, spiegelkabinet, interactie, blik, zichzelf, format, dicht',
 'kind, gemasseerd, geborgenheid, wij, nest, gepassioneerd, mama, beschermen, wou',
 'subjectiviteit, zichzelf, foto, fotografische, naaktheid, poseren, wereld',
 'jezelf, sympathiek, intelligent, narcistisch, mooi, beeld',
 'moslims, onze, wij, individuen, cultuur, vrouwen, moslima, leed, vrouw',
 'culturen, identiteit, dier, thuis, artikel, cultureel']

topic_names = [', '.join(i.split(' - ')) for i in topic_names]
topic_names

['nieuwe wereld',
 'wetenschap, filosofie, kennis, intelligentie, waarheid, wereld',
 'geld, ethisch, systeem, markt, economie, sociaal',
 'liefde, relatie, romantische, feeling, eenzaamheid, leven',
 'facebook, vanavond, sociale mensen, vrienden, delen',
 'muziek, dj, dansen, bewegen, smaak, genre',
 'europa, geschiedenis, landen, history, frankrijk, belgie',
 'kamer, bed, ogen, vuur, oor, zoem',
 'rommel, beelden, orde, norm, sensualiteit',
 'boek, schrijver, lezen, verhaal, mensen',
 'blog, persoonlijke, schrijven, hyperlinks, zin',
 'social, impact, business, innovatie, esg, people, change',
 'stad, cyclists, cars, london, public, space, traffic, transport',
 'reizen, droom, vlieger, avontuur, verhaal, vliegers, hoogtepunt',
 'droom, slaap, denken, nachtelijk',
 'taal, spreken, discours, woorden, vertaalbaarheid, denken',
 'beelden, kunst, taal, foto, representatie, beeldcultuur, artistiek, esthetica',
 'internet, netwerk, spiritueel, online, bereikbaar',
 'empathy, schaamte, geluk

In [62]:
# Creating a groupby object and getting the first list of top_n_words for each topic
grouped = df_pickle.groupby('Topic')['Top_n_words'].first()

# Convert the groupby result into a dictionary
topic_to_top_words = grouped.to_dict()
topic_to_top_words

{-1: 'onze - mensen - wel - wereld - leven - waar - enkel - nieuwe - gaan - gaat',
 0: 'wetenschap - intelligentie - onze - reality - filosofie - dingen - kennis - world - waarheid - wereld',
 1: 'geld - sociale - sociaal - ethische - moeten - economische - economist - systeem - markt - economie',
 2: 'liefde - love - relatie - feel - romantische - feeling - dont - eenzaamheid - leven - would',
 3: 'facebook - fb - bda - vanavond - vrienden - online - sociale - mensen - delen - foto',
 4: 'muziek - dj - music - muzikale - dansen - bewegingen - techno - smaak - genre - nieuwe',
 5: 'europa - frankrijk - europese - land - belgie - duitsland - amerikanen - geschiedenis - landen - history',
 6: 'voelde - kamer - bed - ogen - anders - wit - vuur - oor - zoem - hoor',
 7: 'porno - seks - seksuele - masturberen - rommel - beelden - orde - pornografische - norm - masturbatie',
 8: 'boek - schrijver - schrijven - lezen - verhaal - mensen - gelezen - boeken - leven - moeten',
 9: 'blog - blogs -

In [67]:
i = 0
for name in range(-1, len(topic_names)-1):
    
    topic_to_top_words[name] = topic_names[i] 
    i += 1
    
topic_to_top_words

{-1: 'nieuwe wereld',
 0: 'wetenschap, filosofie, kennis, intelligentie, waarheid, wereld',
 1: 'geld, ethisch, systeem, markt, economie, sociaal',
 2: 'liefde, relatie, romantische, feeling, eenzaamheid, leven',
 3: 'facebook, vanavond, sociale mensen, vrienden, delen',
 4: 'muziek, dj, dansen, bewegen, smaak, genre',
 5: 'europa, geschiedenis, landen, history, frankrijk, belgie',
 6: 'kamer, bed, ogen, vuur, oor, zoem',
 7: 'rommel, beelden, orde, norm, sensualiteit',
 8: 'boek, schrijver, lezen, verhaal, mensen',
 9: 'blog, persoonlijke, schrijven, hyperlinks, zin',
 10: 'social, impact, business, innovatie, esg, people, change',
 11: 'stad, cyclists, cars, london, public, space, traffic, transport',
 12: 'reizen, droom, vlieger, avontuur, verhaal, vliegers, hoogtepunt',
 13: 'droom, slaap, denken, nachtelijk',
 14: 'taal, spreken, discours, woorden, vertaalbaarheid, denken',
 15: 'beelden, kunst, taal, foto, representatie, beeldcultuur, artistiek, esthetica',
 16: 'internet, netwer

In [68]:
# Map the top_n_words to each row in the DataFrame based on its topic
df_pickle['topic_names'] = df_pickle['Topic'].map(topic_to_top_words)

In [71]:
df_pickle.to_pickle("df_topic5.pkl")

In [74]:
centroids = df_pickle.groupby(['Topic','topic_names']).agg({
        '2d_embeddings_x': 'mean',
        '2d_embeddings_y': 'mean'
    }).reset_index()

In [75]:
centroids

,Topic,topic_names,2d_embeddings_x,2d_embeddings_y
0,-1,nieuwe wereld,3.700850,1.085396
1,0,"wetenschap, filosofie, kennis, intelligentie, ...",3.313126,1.158397
2,1,"geld, ethisch, systeem, markt, economie, sociaal",1.774688,-0.874522
3,2,"liefde, relatie, romantische, feeling, eenzaam...",6.412583,-0.795670
4,3,"facebook, vanavond, sociale mensen, vrienden, ...",1.666117,4.582336
5,4,"muziek, dj, dansen, bewegen, smaak, genre",5.804676,4.278247
6,5,"europa, geschiedenis, landen, history, frankri...",0.684549,0.464626
7,6,"kamer, bed, ogen, vuur, oor, zoem",5.810160,1.647412
8,7,"rommel, beelden, orde, norm, sensualiteit",8.197718,0.671203
9,8,"boek, schrijver, lezen, verhaal, mensen",3.793342,2.925212
